In [ ]:
# - Bibliotecas para manipulação e análise de dados
import pandas as pd
import numpy as np
import openpyxl

# - Bibliotecas para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# - Bibliotecas para ML
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

# - Configurações de visualização
%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

print("Bibliotecas importadas com sucesso!")

In [ ]:
# Criando o DataFrame
df = pd.read_excel('teste_contabeis_2024.xlsx')

# Exibindo as primeiras linhas do DataFrame
df.head()